# Защита персональных данных клиентов

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from random import randint

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

In [6]:
data.isna().sum().sum()

0

Я открыл и изучил данные,в колонке 'Страховые выплаты' у нас значения от 0 до 5.Пропусков в данных я не обнаружил.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Подставим в формулу обучения значение P:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y =
$$
$$
= (P^T(X^TX)P)^{-1} X^TP^T y =
$$
$$
= (P^TP)^{-1} (X^TX)^{-1} X^TP^T y =
$$
$$
= P^{-1} (X^TX)^{-1} (P^T)^{-1} X^TP^T y =
$$
$$
= P^{-1} (X^TX)^{-1} EX^T y =
$$
$$
= P^{-1} (X^TX)^{-1} X^T y =
$$
$$
= P^{-1} w
$$

Теперь полученное значение подставим в формулу предсказания:

$$
a' = XPP^{-1}w = Xw = a
$$

**Ответ:** Если умножить признаки на обратимую матрицу качество линейной регрессии не изменится

**Обоснование:** Учитывая свойства матриц я наглядно показал что умножив признаки на обратимую матрицу качество модели не изменится

## Алгоритм преобразования

**Алгоритм**

Исходя из пункта выше я собираюсь создать две модели линейной регресии.В одной модели я не буду домножать признаки на матрицу и посмотрю на результат метрики R2.Во второй модели я разделю выборку на обучающую и валидационную и зашифрую все данные домножив их на обратимую матрицу.Квадратная матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю.Количество строк исходной матрицы должно быть равно количеству колонок матрицы на которую я хочу ее умножить.

**Обоснование**

Исходя из пункта 2 и учитывая свойства матриц мой алгоритм должен сработать

## Проверка алгоритма

In [7]:
#обучу модель линейной регрессии
features = data.drop(['Страховые выплаты'] , axis=1)
target = data['Страховые выплаты']
features_train, features_valid,target_train, target_valid = train_test_split(
    features, target, test_size=0.5,random_state=12345)
model_clean = LinearRegression().fit(features_train,target_train)
predictions = model_clean.predict(features_valid)
score = r2_score(target_valid, predictions)
print('Показания метрики R2 для модели не домноженной на обратимую матрицу',score)

Показания метрики R2 для модели не домноженной на обратимую матрицу 0.4245582177863486


In [8]:
#хочу посмотреть размер исходного датасета
data.shape

(5000, 5)

In [9]:
#хочу превратить данные в матрицу и посмотреть количество строк и колонок
data_matrix = data.values
data_matrix.shape
#в данном случае у меня 5000 строк и 5 колонок,одна из них целевой признак,это значит что мне нужно создать 
#рандомную матрицу с 4 строками и 5000 колонок

(5000, 5)

In [10]:
data_matrix 

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00, 0.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00, 0.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00, 0.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00, 0.00e+00]])

In [11]:
#создаю рандомную матрицу
random_matrix = np.random.randint(0,5,(4,4))
random_matrix

array([[2, 2, 0, 0],
       [4, 3, 0, 2],
       [3, 0, 0, 4],
       [3, 3, 4, 4]])

In [12]:
#добавлено ревьюером

np.linalg.inv(random_matrix)

array([[ 3.   , -2.   ,  1.   ,  0.   ],
       [-2.5  ,  2.   , -1.   , -0.   ],
       [ 1.875, -1.5  ,  0.5  ,  0.25 ],
       [-2.25 ,  1.5  , -0.5  , -0.   ]])

In [13]:
#добавлено ревьюером
#сгенерим квадратную матрицу

square_matrix = np.random.randint(5, size=(4, 4))
square_matrix

array([[0, 1, 0, 4],
       [1, 0, 2, 1],
       [3, 2, 2, 1],
       [4, 2, 0, 1]])

In [14]:
#добавлено ревьюером
#проверим матрицу 4 на 4 на обратимость

np.linalg.inv(square_matrix)

array([[-0.11111111,  0.38888889, -0.38888889,  0.44444444],
       [ 0.11111111, -0.88888889,  0.88888889, -0.44444444],
       [-0.05555556,  0.19444444,  0.30555556, -0.27777778],
       [ 0.22222222,  0.22222222, -0.22222222,  0.11111111]])

In [15]:
#разделяю датасет на признаки и превращаю DataFrame в матрицы
features_matrix = data.drop(['Страховые выплаты'] , axis=1).values
target_matrix = data['Страховые выплаты'].values
#умножаю признаки на мою рандомную матрицу
features_matrix = features_matrix.dot(random_matrix)
features_train_matrix, features_valid_matrix,target_train_matrix, target_valid_matrix = train_test_split(
    features_matrix, target_matrix, test_size=0.5,random_state=12345)
#не удержался и в названии модели оставил отсылку к любимому фильму
model_white_rabbit = LinearRegression().fit(features_train_matrix,target_train_matrix)
predictions_white_rabbit = model_white_rabbit.predict(features_valid_matrix)
score_white_rabbit = r2_score(target_valid_matrix, predictions_white_rabbit)
print('Показания метрики R2 для модели  домноженной на обратимую матрицу',score_white_rabbit)

Показания метрики R2 для модели  домноженной на обратимую матрицу 0.4245582177863092


***Я доказал что если домножить признаки на обратимую матрицу качесвто модели не изменится.***